In [1]:
import pandas as pd
import numpy as np
import altair as alt
import altair_saver

In [2]:
url = 'https://raw.githubusercontent.com/byuidatascience/data4missing/master/data-raw/flights_missing/flights_missing.json'

flights = pd.read_json(url)

flights

# we can use .describe or .value_counts to learn more about the data


flights.describe()

flights.groupby("airport_code").num_of_delays_total.agg("max")

# -999 is one way to code missing data.  Especially social sciences
# What does 1500+ mean?  Greater than 1500?  How do you treat it, because technically some data is missing 
# the value it is about 1500 is not included
# nan is a missing value and n/a
# look for columns that are text but should be numbers.  They may have missing value identifiers like Nan forcing them to be text

# for question 1 calculate likelihood of a disruptive delay, which we would define.  (somewhere between 10-30 minutes I think)

# .pop removed one item from a list for your perusal

flights.assign(
   Year2= lambda x: x.year * 2 
).Year2

0      4010.0
1      4010.0
2      4010.0
3      4010.0
4      4010.0
        ...  
919    4030.0
920    4030.0
921    4030.0
922    4030.0
923    4030.0
Name: Year2, Length: 924, dtype: float64

In [ ]:
# Power Summary Function


# inputs: dataframe and possibly list some specific columns
# outputs would be graphical summary like GGally GGpairs ## and a numerical summary by column (min, max, average, total, STDev, missing values, data type)


def power_summary(df):

    print("This is the Numerical Summary Portion")

    df.describe()

    df.head()

    df.info()

    df.nunique()

    df.count()  #any column with less rows counted has missing or NA values

    # value_counts this is to use on a column.  We would have to iterate through a list of columns and apply it to each
    # maybe iterate through with .min .max .mean and .median  value_counts kind of like the summary function in R
    # df.groupby("sort_column").agg(['max', 'min', 'median'])
    # look at using lambda functions or .assign to simplify the coding of the power_summary function

power_summary(flight)

In [3]:
url2 = "https://github.com/byuidatascience/data4missing/raw/master/data-raw/mtcars_missing/mtcars_missing.json" 

cars = pd.read_json(url2)

cars.groupby("gear").agg(['max', 'min', 'median'])

cars.describe()

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
count,32.000000,32.000000,32.000000,25.000000,32.000000,18.000000,32.000000,26.000000,32.000000,32.000000,32.0000
mean,20.090625,6.187500,230.721875,118.200000,3.596563,3.027500,17.848750,0.500000,0.406250,252.375000,2.8125
std,6.026948,1.785922,123.938694,41.771202,0.534679,0.880909,1.786943,0.509902,0.498991,437.962051,1.6152
min,10.400000,4.000000,71.100000,52.000000,2.760000,1.615000,14.500000,0.000000,0.000000,3.000000,1.0000
25%,15.425000,4.000000,120.825000,93.000000,3.080000,2.395000,16.892500,0.000000,0.000000,3.000000,2.0000
50%,19.200000,6.000000,196.300000,110.000000,3.695000,3.012500,17.710000,0.500000,0.000000,4.000000,2.0000
75%,22.800000,8.000000,326.000000,150.000000,3.920000,3.440000,18.900000,1.000000,1.000000,253.500000,4.0000
max,33.900000,8.000000,472.000000,180.000000,4.930000,5.250000,22.900000,1.000000,1.000000,999.000000,8.0000


In [ ]:
# 2/1/2022 Class Period

# We can choose decimals with round function and specify number of decimal places.

# cars.head()

# cars.cyl.value_counts()

# cars.groupby('cyl').agg(hp_ave = ('hp', np.mean)).round(2)

cyl_data = cars.groupby('cyl').agg(hp_ave = ('hp', np.mean)).reset_index()

# :O is ordinal, :Q is quantitative, :T is time

# when we groupby we turn the grouping variable into the index
# to turn the index back into a regular column we need to add .reset_index() at the end

alt.Chart(cyl_data).mark_point().encode(x = 'cyl:Q', y = alt.Y('hp_ave', axis = alt.Axis(format = '%')))

# for question 2 just drop the rows in month that have n/a's.

flights.query('month != "n/a"')


# we can replace with np.nan to make it an official missing data value

flights.replace([-999, "1500+"], np.nan)

# be wary of unofficial missing values forcing the column to be a string.
# you need to remove or replace the missing value and then reformat the column as the proper data type


# .fillna just fills na's with the value above them in the column  (for the ffill method,  fillna has many methods)
# bfill method fills with the value below the n/a


# from collections import counter  (creates a list of counts)




In [ ]:
# Question 1

# Which airport has the worst delays? Discuss how you chose to define “worst”. 
# Your answer should include a summary table that lists (for each airport) the total number of flights, 
# total number of delayed flights, proportion of delayed flights, and average delay time in hours.


import pandas as pd
import numpy as np
import altair as alt
import altair_saver
from collections import Counter

url_f = 'https://raw.githubusercontent.com/byuidatascience/data4missing/master/data-raw/flights_missing/flights_missing.json'

flights = pd.read_json(url_f)

flights_2 = flights

# Counter(flights.month)

flights.isnull().sum()


flights.groupby('airport_name').minutes_delayed_total.mean()

# flights.groupby("airport_name").filter([]).agg(['sum', 'min', 'median'])
flights_grouped = flights_2.groupby('airport_code')

flights_q1 = flights_grouped.agg(
  Total_Flights = ('num_of_flights_total', sum),
  Total_Delayed_Flights = ('num_of_delays_total', sum),
  Total_Mins_Delay = ('minutes_delayed_total', sum)).assign(
  Perc_of_Flights_Delayed = lambda x: x.Total_Delayed_Flights / x.Total_Flights,
  Mean_Delay_in_Hours = lambda x: ( x.Total_Mins_Delay/ x.Total_Delayed_Flights)/60  
  )

print(flights_q1.to_markdown())


# Question Two

flights_by_month = flights.query('month != "n/a"').groupby('month').agg(
  Total_Flights = ('num_of_flights_total', sum),
  Total_Delayed_Flights = ('num_of_delays_total', sum)).assign(
  Perc_of_Flights_Delayed = lambda x: x.Total_Delayed_Flights / x.Total_Flights
  ).reset_index()

flights_by_month
months = ["January", "Febuary", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
flights_q2_chart = alt.Chart(flights_by_month, title = "% Delayed Flights by Month").mark_bar().encode(x = alt.X('month', sort = months ), y = 'Perc_of_Flights_Delayed')


flights_q2_chart.save('Flights_Q2_Chart.png')


# Question Three

mean_of_delays_late_aircraft = flights.num_of_delays_late_aircraft.replace(-999, np.nan).mean()
flights_q3 = flights
flights_q3.num_of_delays_late_aircraft = flights.num_of_delays_late_aircraft.replace(-999, mean_of_delays_late_aircraft) 

# 100% of delayed flights in the Weather category are due to weather.
# 30% of delayed flights in the Late-Arriving category are due to weather.
# From April to August, 40% of delayed flights in the NAS category are due to weather. The rest of the months, the proportion rises to 65%.


weather = flights_q3.assign(
    severe = lambda x: x.num_of_delays_weather,
    mild_late = (0.3 * flights_q3.num_of_delays_late_aircraft).round(),
    mild_nas = np.where(flights_q3.month == ("April" or "May" or "June" or "July" or "August"), 0.4 * flights_q3.num_of_delays_nas, 0.65 * flights_q3.num_of_delays_nas).round(),
   total_weather = lambda x: x.severe + x.mild_late + x.mild_nas,
).filter(['airport_code','month','severe','mild_late','mild_nas',
   'total_weather', 'num_of_delays_total', 'num_of_flights_total'])


weather_2 = weather.groupby("airport_code").agg(
  Total_Weather_Delays = ('total_weather', sum),
  Total_Flights = ('num_of_flights_total', sum)).assign(total_weather_perc = lambda x: x.Total_Weather_Delays/x.Total_Flights).reset_index()


weather_2_chart = alt.Chart(weather_2, title = "% Delayed Due To Weather").mark_bar().encode(x = 'airport_code', y = alt.Y('total_weather_perc', axis=alt.Axis(format='%')))

weather_2_chart.save('Weather_2_Chart.png')








In [7]:
# Question 5

import pandas as pd
import numpy as np
import altair as alt
import altair_saver
from collections import Counter

url_f = 'https://raw.githubusercontent.com/byuidatascience/data4missing/master/data-raw/flights_missing/flights_missing.json'

flights = pd.read_json(url_f)


flights_clean = flights.replace([-999, "1500+",'n/a', 'nan',"NaN",""], np.NaN)


flights_clean.query('index == 2').head(1).to_json('q_5_json.json')





In [ ]:
# Question 3 Template from class


mean_of_delays_late_aircraft = flights.num_of_delays_late_aircraft.mean()

weather = flights.assign(
    severe = #????,
    mild_late = #????,
    mild_nas = np.where(#????),
   total_weather = # add up severe and mild,
).filter(['airport_code','month','severe','mild_late','mild_nas',
   'total_weather', 'num_of_delays_total']))

# Cars data practice for question 3

# check cars data for missing data

cars.isnull().sum()
mean_of_gear = cars.gear.replace(999, np.nan).mean()

cars_2 = cars
cars_2.gear = cars.gear.replace(999, mean_of_gear)


# Make a new column "bob" that is sum of:
# 100% of cyl, +
# 60% of gear, +
# 100% of carb if disp < 200, but 200% of carb if disp >= 200


answer = cars_2.assign(
  part1 = cars_2.cyl,
  part2 = 0.6 * cars_2.gear,
  part3 = np.where(cars_2.disp < 200, cars_2.carb, 2 * cars_2.carb),
  bob = lambda x: x.part1 + x.part2 + x.part3
)

print(answer.head().filter(["car", "cyl", "disp", "carb", "bob", "part1", "part2", "part3"]).to_markdown())

# Question 4 is aggregate the weather delays data by airport

cars_2.assign(
  part1 = cars_2.cyl,
  part2 = 0.6 * cars_2.gear,
  part3 = np.where(cars_2.disp < 200, cars_2.carb, 2 * cars_2.carb),
  bob = lambda x: x.part1 + x.part2 + x.part3
)